In [5]:
import rustworkx as rx

from itertools import product

from territories import Territory
from territories.partitions import TerritorialUnit, Partition


lyon = TerritorialUnit("Lyon")
marseille = TerritorialUnit("Marseille", es_code="COM:2909") # you can specify an ElasticSearch code
paris = TerritorialUnit("Paris")
nogent = TerritorialUnit("Nogent")
pantin = TerritorialUnit("Pantin")
villeurbane = TerritorialUnit("Villeurbane")
sté = TerritorialUnit("Saint Etienne")

metropole = TerritorialUnit("Grand Lyon", False, Partition.DEP)

sud = TerritorialUnit("Sud", False, Partition.REG)
idf = TerritorialUnit("Île-de-France", False, Partition.REG)
rhone = TerritorialUnit("Rhône", False, Partition.DEP)

france = TerritorialUnit("France", False, Partition.CNTRY)



entities = (france, sud, idf, rhone, metropole, nogent, pantin, paris, marseille, sté, villeurbane, lyon)

tree = rx.PyDiGraph()
entities_indices = tree.add_nodes_from(entities)

mapper = {o : idx for o, idx in zip(entities, entities_indices)}
edges = [
    (france, idf),
    (france, sud),
    
    (idf, nogent),
    (idf, pantin),
    (idf, paris),

    (sud, marseille),
    (sud, rhone),

    (rhone, metropole),
    (rhone, sté),

    (metropole, villeurbane),
    (metropole, lyon),
    ]

tree.add_edges_from([
    (mapper[parent], mapper[child], None) for parent, child in edges
])

Territory.assign_tree(tree)

a = Territory(sté, marseille)
b = Territory(lyon, france)
c = Territory(paris, nogent, pantin, lyon, lyon, metropole)
d = Territory(lyon, villeurbane, marseille)
e = Territory(rhone, idf)
f = Territory(idf, marseille, metropole)

exemples = (a, b, c, d, e, f)

In [5]:
import os
import pickle
import rustworkx as rx

from territories import Partition

from pathlib import Path


path = Path(os.environ["API_CACHE_DIR"], "territorial_tree_state.pickle")

with open(path, "rb") as file:
    _, tree = pickle.load(file)

# comms = [t.es_code + '\n' for t in tree.nodes() if t.partition_type == Partition.COM]

# with open("comms.txt", "w") as file:
#     file.writelines(comms)

In [34]:
next((node.name, node.es_code) for node in tree.nodes() if "Nogent-sur-M" in node.name)

('Nogent-sur-Marne', 'COM:94052')

In [20]:
root = next(i for i in tree.node_indices() if tree.in_degree(i) == 0)
for dep in tree.successors(root):
    print(dep, dep.es_code)

Mayotte REG:06
La Réunion REG:04
Guyane REG:03
Normandie REG:28
Hauts-de-France REG:32
Provence-Alpes-Côte d'Azur REG:93
Bretagne REG:53
Corse REG:94
Occitanie REG:76
Grand Est REG:44
Pays de la Loire REG:52
Île-de-France REG:11
Bourgogne-Franche-Comté REG:27
Auvergne-Rhône-Alpes REG:84
Nouvelle-Aquitaine REG:75
Centre-Val de Loire REG:24
Martinique REG:02
Guadeloupe REG:01


In [37]:
reg = next(node for node in tree.nodes() if node.es_code == "REG:28")
for dep in tree.successors(reg.tree_id):
    print(dep, dep.es_code)

Orne DEP:61
Manche DEP:50
Calvados DEP:14
Eure DEP:27
Seine-Maritime DEP:76


In [4]:
cols = ('id', 'parent_id', 'label', 'level')

extract = lambda x: 'null' if len(x) < 1 else x[0].es_code
format_line = lambda x: '; '.join(x) + '\n'

with open("tree_large.txt", "w") as file:
    gen = ((n.es_code, extract(tree.predecessors(n.tree_id)), n.name, str(n.partition_type)) for n in tree.nodes())
    file.writelines((format_line(line) for line in gen))